In [1]:
import torch
%load_ext autoreload
%autoreload 2

In [2]:
from model.threedepn import ThreeDEPNDecoder
from util.model import summarize_model

threedepn = ThreeDEPNDecoder()
print(summarize_model(threedepn))

   | Name         | Type             | Params  
-----------------------------------------------------
0  | bottleneck   | Sequential       | 263168  
1  | bottleneck.0 | Linear           | 65792   
2  | bottleneck.1 | ReLU             | 0       
3  | bottleneck.2 | Linear           | 65792   
4  | bottleneck.3 | ReLU             | 0       
5  | bottleneck.4 | Linear           | 131584  
6  | bottleneck.5 | ReLU             | 0       
7  | decoder1     | Sequential       | 8389376 
8  | decoder1.0   | ConvTranspose3d  | 8388864 
9  | decoder1.1   | BatchNorm3d      | 512     
10 | decoder1.2   | ReLU             | 0       
11 | decoder2     | Sequential       | 2097536 
12 | decoder2.0   | ConvTranspose3d  | 2097280 
13 | decoder2.1   | BatchNorm3d      | 256     
14 | decoder2.2   | ReLU             | 0       
15 | decoder3     | Sequential       | 524480  
16 | decoder3.0   | ConvTranspose3d  | 524352  
17 | decoder3.1   | BatchNorm3d      | 128     
18 | decoder3.2   | ReLU          

In [36]:
from data.shapenet import ShapeNet

# Create a dataset with train split
train_dataset = ShapeNet('train', filter_class = 'airplane')
overfit_dataset = ShapeNet('overfit')

# Get length, which is a call to __len__ function
print(f'Length of train set: {len(train_dataset)}')  # expected output: 153540
# Get length, which is a call to __len__ function
print(f'Length of overfit set: {len(overfit_dataset)}')  # expected output: 64

Length of train set: 4045
Length of overfit set: 64


In [17]:
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes

sample = train_dataset[101]
print(f'Target DF: {sample["target_df"].shape}')  # expected output: (32, 32, 32)

input_mesh = marching_cubes(sample['target_df'], level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

Target DF: (32, 32, 32)


Output()

In [5]:
from training import train
config = {
    'experiment_name': 'overfit',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': True,
    'batch_size': 32,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 4,
    'print_every_n': 1,
    'validate_every_n': 250,
    'latent_code_length' : 256,
    'vad_free' : True,
    'test': False,
    'kl_weight': 1,
    'resume_ckpt': None

}
train.main(config)

Using device: cuda:0
Loading saved model, latent codes, and latent variances...
Training params: 3
[001/00004] train_loss: 0.010115 kl_loss: 0.007572 normal_loss: 0.002543
[001/00004] train_loss: 0.010374 kl_loss: 0.007749 normal_loss: 0.002626
[002/00004] train_loss: 0.008716 kl_loss: 0.007542 normal_loss: 0.001174
[002/00004] train_loss: 0.008393 kl_loss: 0.007422 normal_loss: 0.000971
[003/00004] train_loss: 0.008175 kl_loss: 0.007302 normal_loss: 0.000872
[003/00004] train_loss: 0.008076 kl_loss: 0.007325 normal_loss: 0.000750
[004/00004] train_loss: 0.007750 kl_loss: 0.007125 normal_loss: 0.000625
[004/00004] train_loss: 0.007689 kl_loss: 0.007174 normal_loss: 0.000515


In [6]:
from training import train
config = {
    'experiment_name': 'overfit_test',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': True,
    'batch_size': 32,
    'resume_ckpt': None,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 2,
    'print_every_n': 1,
    'validate_every_n': 250,
    'latent_code_length' : 256,
    'vad_free' : True,
    'test': True,
    'kl_weight': 1,
    'resume_ckpt': 'overfit_train'

}
train.main(config)

Using CPU
Loading saved model, latent codes, and latent variances...
Training params: 2
[001/00002] train_loss: 0.009291 kl_loss: 0.009158 normal_loss: 0.000133
[002/00002] train_loss: 0.009056 kl_loss: 0.008923 normal_loss: 0.000134


In [1]:
from training import train
config = {
    'experiment_name': 'generalization_airplane_vad_deeper',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 32,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'validate_every_n': 250,
    'latent_code_length' : 256,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.01,
    'resume_ckpt': None,
    'filter_class': 'airplane',
    'decoder_var' : True
}
train.main(config)

Using device: cuda:0
Training params: 3
[000/00099] train_loss: 0.113204 kl_loss: 0.500511 normal_loss: 0.108199
[001/00072] train_loss: 0.047127 kl_loss: 0.467388 normal_loss: 0.042453
[002/00045] train_loss: 0.041506 kl_loss: 0.430261 normal_loss: 0.037203
[003/00018] train_loss: 0.037069 kl_loss: 0.396226 normal_loss: 0.033107
[003/00118] train_loss: 0.035432 kl_loss: 0.364296 normal_loss: 0.031789
[004/00091] train_loss: 0.033546 kl_loss: 0.342165 normal_loss: 0.030125
[005/00064] train_loss: 0.032608 kl_loss: 0.332488 normal_loss: 0.029283
[006/00037] train_loss: 0.030671 kl_loss: 0.321053 normal_loss: 0.027461
[007/00010] train_loss: 0.031114 kl_loss: 0.307378 normal_loss: 0.028040
[007/00110] train_loss: 0.029205 kl_loss: 0.293948 normal_loss: 0.026265
[008/00083] train_loss: 0.029286 kl_loss: 0.278934 normal_loss: 0.026496
[009/00056] train_loss: 0.031165 kl_loss: 0.268189 normal_loss: 0.028483
[010/00029] train_loss: 0.033534 kl_loss: 0.255888 normal_loss: 0.030975
[011/00002]

[088/00123] train_loss: 0.017917 kl_loss: 0.073647 normal_loss: 0.017181
[089/00096] train_loss: 0.017808 kl_loss: 0.072722 normal_loss: 0.017081
[090/00069] train_loss: 0.017798 kl_loss: 0.073440 normal_loss: 0.017063
[091/00042] train_loss: 0.018076 kl_loss: 0.074233 normal_loss: 0.017334
[092/00015] train_loss: 0.017761 kl_loss: 0.072420 normal_loss: 0.017037
[092/00115] train_loss: 0.017928 kl_loss: 0.072796 normal_loss: 0.017200
[093/00088] train_loss: 0.017804 kl_loss: 0.073089 normal_loss: 0.017073
[094/00061] train_loss: 0.017892 kl_loss: 0.072936 normal_loss: 0.017162
[095/00034] train_loss: 0.017920 kl_loss: 0.070905 normal_loss: 0.017211
[096/00007] train_loss: 0.017763 kl_loss: 0.073296 normal_loss: 0.017030
[096/00107] train_loss: 0.017951 kl_loss: 0.072174 normal_loss: 0.017229
[097/00080] train_loss: 0.017646 kl_loss: 0.071963 normal_loss: 0.016926
[098/00053] train_loss: 0.018194 kl_loss: 0.072043 normal_loss: 0.017474
[099/00026] train_loss: 0.017738 kl_loss: 0.071641 

[177/00120] train_loss: 0.017386 kl_loss: 0.068216 normal_loss: 0.016704
[178/00093] train_loss: 0.017285 kl_loss: 0.068315 normal_loss: 0.016602
[179/00066] train_loss: 0.017466 kl_loss: 0.066803 normal_loss: 0.016798
[180/00039] train_loss: 0.017578 kl_loss: 0.068516 normal_loss: 0.016893
[181/00012] train_loss: 0.017308 kl_loss: 0.067903 normal_loss: 0.016629
[181/00112] train_loss: 0.017378 kl_loss: 0.067897 normal_loss: 0.016700
[182/00085] train_loss: 0.017435 kl_loss: 0.066644 normal_loss: 0.016768
[183/00058] train_loss: 0.017426 kl_loss: 0.068389 normal_loss: 0.016743
[184/00031] train_loss: 0.017402 kl_loss: 0.067653 normal_loss: 0.016726
[185/00004] train_loss: 0.017537 kl_loss: 0.068059 normal_loss: 0.016856
[185/00104] train_loss: 0.017496 kl_loss: 0.067481 normal_loss: 0.016822
[186/00077] train_loss: 0.017286 kl_loss: 0.067342 normal_loss: 0.016612
[187/00050] train_loss: 0.017214 kl_loss: 0.067333 normal_loss: 0.016541
[188/00023] train_loss: 0.017648 kl_loss: 0.068016 

[266/00117] train_loss: 0.017354 kl_loss: 0.067723 normal_loss: 0.016677
[267/00090] train_loss: 0.017493 kl_loss: 0.067932 normal_loss: 0.016814
[268/00063] train_loss: 0.017315 kl_loss: 0.067601 normal_loss: 0.016639
[269/00036] train_loss: 0.017308 kl_loss: 0.066964 normal_loss: 0.016638
[270/00009] train_loss: 0.017504 kl_loss: 0.067672 normal_loss: 0.016827
[270/00109] train_loss: 0.017456 kl_loss: 0.067879 normal_loss: 0.016777
[271/00082] train_loss: 0.017513 kl_loss: 0.067897 normal_loss: 0.016834
[272/00055] train_loss: 0.017239 kl_loss: 0.066431 normal_loss: 0.016575
[273/00028] train_loss: 0.017445 kl_loss: 0.069203 normal_loss: 0.016753
[274/00001] train_loss: 0.017517 kl_loss: 0.066355 normal_loss: 0.016854
[274/00101] train_loss: 0.017507 kl_loss: 0.067196 normal_loss: 0.016835


KeyboardInterrupt: 

In [97]:
from training import train
config = {
    'experiment_name': 'generalization_airplane_vad_test',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 32,
    'learning_rate_model': 0.001,
    'learning_rate_code': 0.001,
    'learning_rate_log_var':0.001,
    'max_epochs': 500,
    'print_every_n': 100,
    'validate_every_n': 250,
    'latent_code_length' : 256,
    'vad_free' : True,
    'test': True,
    'kl_weight': 0.1,
    'resume_ckpt': 'generalization_airplane_vad',
    'filter_class': 'airplane',
    'decoder_var' : True
}
train.main(config)

Using device: cuda:0
Loading saved model, latent codes, and latent variances...
Training params: 2
[000/00099] train_loss: 0.017962 kl_loss: 0.014923 normal_loss: 0.016470
[001/00072] train_loss: 0.017872 kl_loss: 0.014774 normal_loss: 0.016395
[002/00045] train_loss: 0.017851 kl_loss: 0.014575 normal_loss: 0.016393
[003/00018] train_loss: 0.017669 kl_loss: 0.014604 normal_loss: 0.016208
[003/00118] train_loss: 0.017860 kl_loss: 0.014563 normal_loss: 0.016404
[004/00091] train_loss: 0.017988 kl_loss: 0.014671 normal_loss: 0.016521
[005/00064] train_loss: 0.017824 kl_loss: 0.014523 normal_loss: 0.016372
[006/00037] train_loss: 0.017844 kl_loss: 0.014805 normal_loss: 0.016364
[007/00010] train_loss: 0.018040 kl_loss: 0.014678 normal_loss: 0.016572
[007/00110] train_loss: 0.017887 kl_loss: 0.014810 normal_loss: 0.016406
[008/00083] train_loss: 0.017823 kl_loss: 0.014727 normal_loss: 0.016350
[009/00056] train_loss: 0.018025 kl_loss: 0.014937 normal_loss: 0.016531
[010/00029] train_loss: 0

KeyboardInterrupt: 

In [75]:
from model.threedepn import ThreeDEPNDecoder
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes
import torch.distributions as dist
import torch

# load model
name_temp = 'generalization_airplane_vad_lowkl'
model = ThreeDEPNDecoder()
model.load_state_dict(torch.load(f"runs/{name_temp}/model_best.ckpt", map_location='cpu'))
latent_vectors = torch.load(f"runs/{name_temp}/latent_best.pt", map_location = 'cpu')
log_vars = torch.load(f"runs/{name_temp}/log_var_best.pt", map_location = 'cpu')
Dist = dist.Normal(latent_vectors[1002], torch.exp(log_vars[1002]))
Dist2 = dist.Normal(latent_vectors[1000], torch.exp(log_vars[1000]))
#x_vad_n = torch.randn(256, device='cpu')
x_vad = Dist.rsample()#.unsqueeze(0)
x_vad2 = Dist2.rsample()#.unsqueeze(0)

a1 = 0.5
a2 = 0.5
f1 = 5
f2 = 105
output_meshes_int = model((x_vad*a1 + x_vad2*a2).unsqueeze(0))
output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

output_meshes = model(x_vad.unsqueeze(0))
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)

output_meshes = model(x_vad2.unsqueeze(0))
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)


Output()

Output()

Output()

In [62]:
from model.threedepn import ThreeDEPNDecoder
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes
import torch.distributions as dist
import torch
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes

sample = train_dataset[5]
print(f'Target DF: {sample["target_df"].shape}')  # expected output: (32, 32, 32)

input_mesh = marching_cubes(sample['target_df'], level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)
# load model
name_temp = 'generalization_airplane_no_vad'
model = ThreeDEPNDecoder()
model.load_state_dict(torch.load(f"runs/{name_temp}/model_best.ckpt", map_location='cpu'))

# load latent codes and latent variances
latent_vectors = torch.load(f"runs/{name_temp}/latent_best.pt", map_location = 'cpu')
log_vars = torch.load(f"runs/{name_temp}/log_var_best.pt", map_location = 'cpu')

x_vad_n = x_vad_n.unsqueeze(0)
output_meshes = model(latent_vectors[5].unsqueeze(0))

# Visualize
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)

output_meshes = model(latent_vectors[105].unsqueeze(0))

# Visualize
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)
a1 = 0.5
a2 = 0.5
f1 = 5
f2 = 105
output_meshes_int = model((latent_vectors[f1]*a1 + latent_vectors[f2]*a2).unsqueeze(0))
output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

Target DF: (32, 32, 32)


Output()

Output()

Output()

Output()